In [133]:
import os
import glob
import re
import statistics
from pathlib import Path
import pandas as pd
import tiktoken

directory = Path("../text/KaggleTilak")
glossary = directory/'kaggla_tilak_glossary.txt'
stories = directory/'kaggle_tilak_mahabharata.txt'

In [108]:
directory = Path('../text/KaggleTilak')
file = directory/'1-18 books combined.txt'


In [129]:


encoder_name = "cl100k_base"
encoding = tiktoken.get_encoding(encoder_name)

def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# n_tokens = num_tokens_from_string(string = text, encoding_name = encoder_name)
# 0.0001*n_tokens/1000

In [109]:

book_text = ""
current_book = "NA"
outfile = ""
book_number = 0

with open(file) as books:
     for line in books:
          ## Match start of a book
          if re.match(r".{1,20}Parva *$", line):
               ## Skip initial empty text
               if not (current_book == "NA"):
                    outfile = directory/f"books/{book_number}_{current_book.replace(' ', '_')}.txt"
                    outfile.write_text(book_text.strip())

               # print("Parva", line)
               current_book = line.replace("\n", "").strip()
               book_number += 1
               book_text = ""
               continue
          book_text = book_text + "\n" + line

     ## Last book
     outfile = directory/f"books/{book_number}_{current_book.replace(' ', '_')}.txt"
     outfile.write_text(book_text.strip())



In [164]:
subdirectory = directory/'books'
file_list = glob.glob(f"{subdirectory}/*.txt")

data = []

for file_name in file_list:

     file = file_name.split("/")[-1]
     book_number = file[:len(file)-4].split('_')[0]
     book_name = " ".join(file[:len(file)-4].split('_')[1:3])

     next_line = "Chapter Heading"
     current_chapter = "NA"
     commentary = False

     with open(file_name) as book:
          for line in book:
               if not line.strip():
                    # print("___Empty Line___")
                    continue
          
               ## Match Chapter
               if re.match(r"^(Chapter) (?!Commentary).* *$", line):
                    # print("Chapter", line)
                    current_chapter = line.replace("\n", "").strip()
                    commentary = False
                    next_line = 'Chapter Heading'
                    continue

               ## Match chapter heading
               if next_line == "Chapter Heading":
                    # print("Chapter Heading ->", line)
                    current_chapter_heading = line
                    next_line = "NA"
                    continue

               if re.match(r"^(Chapter) (Commentary).* *$", line):
                    commentary = True
               
               data = data + [[book_number, book_name, current_chapter, current_chapter_heading, commentary, line]]


df_lines = pd.DataFrame(data, columns=['book_number', 'book_name', 'current_chapter', 'current_chapter_heading', 'commentary', 'text'])
df_lines['book_number'] = df_lines['book_number'].astype('int')
df_lines['tokens'] = df_lines['text'].apply( num_tokens_from_string )


# statistics.mean(df_splits.tokens)

In [165]:

df_chapters = df_lines.groupby(
    ['book_number', 'book_name', 'current_chapter', 'current_chapter_heading', 'commentary']
    ).agg({'text': ' '.join}).reset_index()
df_chapters['book_number'] = df_chapters['book_number'].astype('int')
df_chapters['tokens'] = df_chapters['text'].apply( num_tokens_from_string )


In [166]:
df_lines.to_csv(subdirectory/"complete_text_lines.csv")
df_chapters.to_csv(subdirectory/"complete_text_chapters.csv")


In [168]:
statistics.mean(df_chapters['tokens'])

2548.2115384615386